In [1]:
raw_data = sc.textFile('hdfs://master:54310/unique_not_stemmed')

In [2]:
import json
data = raw_data.map(json.loads)

In [43]:
labels = data.map(lambda doc: doc['Labels'])
features = data.map(lambda doc: doc['Features'])

In [44]:
labels_b = labels.flatMap(lambda d: d).distinct()
features_b = features.flatMap(lambda d: d).distinct()

In [45]:
features_b.take(6)

['меморизация', 'testovich', 'стройторги', 'пептидных', 'изумил', 'vanil']

In [5]:
import re

In [22]:
def is_russian(w: str):
    return bool(re.match('.*([а-я]).*', w))

In [37]:
def is_not_russian(w: str):
    return not is_russian(w)

In [34]:
is_russian('')

False

In [46]:
eng_labels = labels_b.filter(is_not_russian)
eng_features = features_b.filter(is_not_russian)

In [47]:
eng_labels.count()

53458

In [48]:
eng_features.count()

314448

In [57]:
labels_w = labels.flatMap(lambda d: d).filter(is_not_russian).map(lambda l: (l,1)). \
    reduceByKey(int.__add__).sortBy(lambda l: l[1], ascending=False). \
    map(lambda l: l[0]).take(5000)

In [58]:
features_w = features.flatMap(lambda d: d).filter(is_not_russian).map(lambda l: (l,1)). \
    reduceByKey(int.__add__).sortBy(lambda l: l[1], ascending=False). \
    map(lambda l: l[0]).take(5000)

In [59]:
with open('/home/hadoop/data/eng_labels.txt', 'w+') as fp:
    for w in labels_w:
        fp.write(w+'\n')

In [61]:
with open('/home/hadoop/data/eng_features.txt', 'w+') as fp:
    for w in features_w:
        fp.write(w+'\n')

In [63]:
with open('/home/hadoop/data/eng_all.txt', 'w+') as fp:
    for w in sorted(set(features_w+labels_w)):
        fp.write(w+'\n')